In [ ]:
import torch
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [ ]:
df = pd.read_csv("../data/cleaned/hcmhouseprice_filtered.csv")

# onehot
address_dummies = pd.get_dummies(df['address']).astype(int)

feature_cols = ['area', 'bedrooms', 'bathrooms']
target_col = 'price'

scaler_area = StandardScaler()
scaler_price = StandardScaler()

df['area'] = scaler_area.fit_transform(df[['area']])
df['price'] = scaler_price.fit_transform(df[['price']])

X_df = pd.concat([df[feature_cols], address_dummies], axis=1)

X = torch.tensor(X_df.values, dtype=torch.float32)
y = torch.tensor(df[[target_col]].values, dtype=torch.float32)

In [ ]:
model = torch.nn.Linear(in_features=X.shape[1], out_features=1)

In [ ]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
epochs = 20000
for epoch in range(epochs):
    y_pred = model(X)
    loss = criterion(y_pred, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.6f}")

In [ ]:
w = model.weight.data.numpy().flatten()
b = model.bias.item()

for name, weight in zip(X_df.columns, w):
    print(f"{name:25s}: {weight: .4f}")

print(f"\nBias (intercept): {b:.4f}")